# Exergoeconomic Optimization Model

## Objective Function
$$\min F_{obj} = \sum_k \dot{Z}_k^{cap} + \sum_k \dot{C}_{D,k}$$

Where:
- $\dot{Z}_k^{cap}$ = Annualized capital cost rate (\$/h)
- $\dot{C}_{D,k}$ = Exergy destruction cost rate (\$/h)

In [117]:
using Pkg
Pkg.activate("Exergy")
Pkg.status()

Status `D:\ترم هفت\Exergy_proj\Exergy\Project.toml`


  Activating project at `d:\ترم هفت\Exergy_proj\Exergy`


  [336ed68f] CSV v0.10.15
  [a93c6f00] DataFrames v1.8.1
  [87dc4568] HiGHS v1.20.1
⌃ [7073ff75] IJulia v1.33.0 [loaded: v1.31.1]
  [4076af6c] JuMP v1.29.4
  [0db19996] NBInclude v2.4.0
  [fdbf4ff8] XLSX v0.10.4
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [118]:
using NBInclude
using JuMP
using HiGHS
using CSV
using DataFrames
using Printf

## Load Technology Modules

In [119]:
@nbinclude("grid_electricity.ipynb")
@nbinclude("solar_pv.ipynb")
@nbinclude("wind_turbine.ipynb")
@nbinclude("battery_storage.ipynb")
@nbinclude("gas_furnace.ipynb")
@nbinclude("heat_pump.ipynb")
@nbinclude("pcm_storage.ipynb")
@nbinclude("solar_thermal.ipynb")

Running Grid Electricity Tests...
Test Summary: | Pass  Total  Time
Parameters    |    4      4  0.0s
Test Summary:  | Pass  Total  Time
Energy Balance |    2      2  0.0s
Test Summary:  | Pass  Total  Time
Exergy Balance |    2      2  0.0s
Test Summary: | Pass  Total  Time
Mass Flow     |    1      1  0.0s
Test Summary: | Pass  Total  Time
Full Analysis |    3      3  0.0s
All tests passed!
Grid Electricity Analysis
Home demand:        5.0 kW
Fuel energy input:  13.16 kW
Fuel exergy input:  13.95 kW
Exergy destruction: 8.95 kW
Fuel mass rate:     0.9474 kg/h
Energy efficiency:  38.0%
Exergy efficiency:  35.8%
Running Solar PV Tests...
Test Summary:       | Pass  Total  Time
Solar Exergy Factor |    2      2  0.0s
Test Summary:    | Pass  Total  Time
Cell Temperature |    2      2  0.0s
Test Summary:      | Pass  Total  Time
Temperature Effect |    2      2  0.0s
Test Summary: | Pass  Total  Time
Power Output  |    2      2  0.0s
Test Summary:   | Pass  Total  Time
Zero Irradiance |  

## Cost Data Tables

### Capital Costs

In [120]:
# Capital Cost Data (USD)
capital_costs = DataFrame(
    Technology = ["Solar PV", "Wind Turbine", "Battery", "Gas Furnace",
                  "Heat Pump", "PCM Storage", "Solar Thermal"],
    Unit = ["\$/kW", "\$/kW", "\$/kWh", "\$/kW_th",
            "\$/kW_th", "\$/kWh_th", "\$/m²"],
    Cost = [
        2900.0,   # Residential PV installed cost ≈ 2.90 $/Wdc (Q1 2023)
        7370.0,   # Small wind installed cost (2023 average)
        1152.0,   # Derived from NREL PV-only vs PV+storage delta (see note)
        117.0,    # Derived from NESCAUM Table 17: $2233 for 65kBtu/h (~19.05 kWth)
        429.0,    # Derived from NESCAUM Table 17: $8174 for 65kBtu/h (~19.05 kWth)
        30.0,     # Representative mid value within literature range (12–57.3 $/kWh_th)
        1000.0    # Household-sized solar water heater order-of-magnitude cost
    ],
    Lifetime_years = [25, 20, 10, 20, 15, 20, 25],
    Source = [
        "NREL PV+Storage Cost Benchmark (Q1 2023)",
        "PNNL Distributed Wind Market Report 2024 Edition",
        "NREL PV+Storage Cost Benchmark (Q1 2023) — derived from PV-only vs PV+storage",
        "NESCAUM Heat Pumps in the Northeast & Mid-Atlantic (Table 17) — derived to \$/kW_th",
        "NESCAUM Heat Pumps in the Northeast & Mid-Atlantic (Table 17) — derived to \$/kW_th",
        "Elfeky et al., Energy (2022) — LHS with PCM cost range",
        "DOE Energy Saver — solar water heater cost estimate"
    ]
)

println("=== Capital Costs ===")
capital_costs

=== Capital Costs ===


Row,Technology,Unit,Cost,Lifetime_years,Source
,String,String,Float64,Int64,String
1,Solar PV,$/kW,2900.0,25,NREL PV+Storage Cost Benchmark (Q1 2023)
2,Wind Turbine,$/kW,7370.0,20,PNNL Distributed Wind Market Report 2024 Edition
3,Battery,$/kWh,1152.0,10,NREL PV+Storage Cost Benchmark (Q1 2023) — derived from PV-only vs PV+storage
4,Gas Furnace,$/kW_th,117.0,20,NESCAUM Heat Pumps in the Northeast & Mid-Atlantic (Table 17) — derived to $/kW_th
5,Heat Pump,$/kW_th,429.0,15,NESCAUM Heat Pumps in the Northeast & Mid-Atlantic (Table 17) — derived to $/kW_th
6,PCM Storage,$/kWh_th,30.0,20,"Elfeky et al., Energy (2022) — LHS with PCM cost range"
7,Solar Thermal,$/m²,1000.0,25,DOE Energy Saver — solar water heater cost estimate


### Fuel and Electricity Costs

In [121]:
# Fuel and Energy Costs
fuel_costs = DataFrame(
    Fuel = ["Natural Gas", "Grid Electricity"],
    Energy_Cost = [0.012, 0.12],  # $/kWh (energy basis)
    Unit = ["\$/kWh", "\$/kWh"],
    LHV_or_Note = ["47.1 MJ/kg", "Delivered"],
    Source = ["EIA", "EIA"]
)
println("=== Fuel Costs (Energy Basis) ===")
fuel_costs

=== Fuel Costs (Energy Basis) ===


Row,Fuel,Energy_Cost,Unit,LHV_or_Note,Source
,String,Float64,String,String,String
1,Natural Gas,0.012,$/kWh,47.1 MJ/kg,EIA
2,Grid Electricity,0.12,$/kWh,Delivered,EIA


### Exergy Costs for Fuels

**Natural Gas Chemical Exergy:**
$$\psi_{NG} = 1.06 \cdot LHV$$

**Grid Electricity from Power Plant (Brayton Cycle):**
$$c_{ex,fuel} = \frac{c_{en,fuel}}{\psi} = \frac{c_{NG}}{1.06}$$

In [122]:
# Exergy Cost Calculation
const ψ_NG = 1.06  # Chemical exergy factor for natural gas
const c_NG_energy = 0.012  # $/kWh (energy basis)
const c_NG_exergy = c_NG_energy / ψ_NG  # $/kWh (exergy basis)

# Grid electricity exergy cost (includes power plant inefficiency)
const η_powerplant = 0.40  # Brayton cycle efficiency
const c_grid_exergy = c_NG_exergy / η_powerplant  # $/kWh_ex

exergy_costs = DataFrame(
    Source = ["Natural Gas", "Grid Electricity"],
    Energy_Cost = [c_NG_energy, 0.12],
    Exergy_Factor = [ψ_NG, 1.0],
    Exergy_Cost = [c_NG_exergy, c_grid_exergy],
    Unit = ["\$/kWh_ex", "\$/kWh_ex"],
    Note = ["Chemical exergy", "Including 40% plant efficiency"]
)
println("=== Exergy Costs ===")
exergy_costs

=== Exergy Costs ===


Row,Source,Energy_Cost,Exergy_Factor,Exergy_Cost,Unit,Note
,String,Float64,Float64,Float64,String,String
1,Natural Gas,0.012,1.06,0.0113208,$/kWh_ex,Chemical exergy
2,Grid Electricity,0.12,1.0,0.0283019,$/kWh_ex,Including 40% plant efficiency


### O&M Costs

In [123]:
# O&M Costs (% of capital per year)
om_costs = DataFrame(
    Technology = ["Solar PV", "Wind Turbine", "Battery", "Gas Furnace",
                  "Heat Pump", "PCM Storage", "Solar Thermal"],
    Fixed_OM_pct = [1.0, 2.0, 1.0, 2.0, 2.0, 0.5, 1.5],
    Variable_OM = [0.0, 0.0, 0.002, 0.001, 0.001, 0.0, 0.0],
    Variable_Unit = ["N/A", "N/A", "\$/kWh", "\$/kWh_th", "\$/kWh_th", "N/A", "N/A"]
)
println("=== O&M Costs ===")
om_costs

=== O&M Costs ===


Row,Technology,Fixed_OM_pct,Variable_OM,Variable_Unit
,String,Float64,Float64,String
1,Solar PV,1.0,0.0,N/A
2,Wind Turbine,2.0,0.0,N/A
3,Battery,1.0,0.002,$/kWh
4,Gas Furnace,2.0,0.001,$/kWh_th
5,Heat Pump,2.0,0.001,$/kWh_th
6,PCM Storage,0.5,0.0,N/A
7,Solar Thermal,1.5,0.0,N/A


## Load Input Data

In [124]:
# Load demand and weather data
data = CSV.read("data_for_model.csv", DataFrame)
# Remove empty rows
data = dropmissing(data)
T = nrow(data)  # Number of time steps
Δt = 1.0  # hours

println("Loaded $T hourly time steps")
first(data, 5)

Loaded 8760 hourly time steps


Row,timestamp,electricity_demand,cooling_demand,heating_demand,solar_irradiance,wind_speed,temperature,db_temp,wb_temp
,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1/1/2002 1:00,0.320316,0.0,3.21427,0.0,3.13,8.32,8.32,5.88
2,1/1/2002 2:00,0.320486,0.0,3.57527,0.0,2.58,8.36,8.36,5.66
3,1/1/2002 3:00,0.320564,0.0,3.5653,0.0,1.87,8.47,8.47,5.54
4,1/1/2002 4:00,0.320665,0.0,3.66996,0.0,1.51,8.57,8.57,5.49
5,1/1/2002 5:00,0.336091,0.0,10.5712,0.0,1.83,8.12,8.12,5.39


## System Parameters

In [125]:
# Reference environment (using different names to avoid conflict with included modules)
const T_ref = 298.15  # K (25°C reference dead state)
const T_indoor_set = 293.15  # K (20°C indoor setpoint)

# Economic parameters
const r = 0.08  # Discount rate
const hours_per_year = 8760

# Capital Recovery Factor
CRF(n) = r * (1+r)^n / ((1+r)^n - 1)

# Annualized cost rate ($/h)
annualized_rate(capex, n) = capex * CRF(n) / hours_per_year

println("CRF for 20 years: ", round(CRF(20), digits=4))

CRF for 20 years: 0.1019


## Technology Instances

In [126]:
# ============================================================
# MODEL PARAMETERS (not decision variables)
# ============================================================

# These are constants used in the optimization, NOT to be optimized
# The actual capacities (Cap_pv, Cap_wt, etc.) are decision variables

# Grid (existing Brayton cycle power plant)
grid = GridElectricity()  # For exergy calculations only

# Furnace (existing - NO CAPITAL COST)
# Uses natural gas at $0.012/kWh, η_AFUE = 95%

# Heat Pump (existing - NO CAPITAL COST)  
# Uses electricity, COP depends on temperature

# Rooftop constraint
const A_roof_max = 60.0  # m² available rooftop area

# Wind turbine - fixed swept area for exergy calculation
# Even if turbine not selected, wind kinetic energy is available
const A_swept_max = 50.0  # m² maximum swept area (corresponds to ~10kW turbine)
const D_rotor = sqrt(4 * A_swept_max / π)  # Rotor diameter [m]

# ============================================================
# EMISSIONS FACTORS
# ============================================================
# Natural gas combustion: ~0.181 kg CO2 per kWh of fuel (LHV basis)
# Source: EPA, EIA - natural gas emission factor
const EF_NG = 0.181  # kg CO2 / kWh_fuel

# Grid electricity emissions (from NG power plant)
# = EF_NG / η_plant (accounts for power plant inefficiency)
const EF_grid = EF_NG / η_powerplant  # kg CO2 / kWh_elec

println("Model Parameters:")
println("─"^50)
println("EXISTING INFRASTRUCTURE (no capital cost):")
println("  Grid:    Brayton η = $(grid.η_en*100)%")
println("  Furnace: AFUE = 95%, fuel = \$0.012/kWh")
println("  Heat Pump: COP = f(ΔT), existing installation")
println("─"^50)
println("NEW INVESTMENTS (capacity to be optimized):")
println("  Solar PV:      @ \$1000/kW  (rooftop limited)")
println("  Wind Turbine:  0-50 kW @ \$1500/kW")
println("  Battery:       0-200 kWh @ \$300/kWh")
println("  Solar Thermal: @ \$200/m²  (rooftop limited)")
println("─"^50)
println("CONSTRAINTS:")
println("  Rooftop Area: PV + Solar Thermal ≤ $(A_roof_max) m²")
println("  Wind Swept Area (for exergy): $(A_swept_max) m² (D = $(round(D_rotor, digits=1)) m)")
println("─"^50)
println("EMISSIONS FACTORS:")
@printf("  Natural Gas:      %.3f kg CO₂/kWh_fuel\n", EF_NG)
@printf("  Grid Electricity: %.3f kg CO₂/kWh_elec (η=%.0f%%)\n", EF_grid, η_powerplant*100)

Model Parameters:
──────────────────────────────────────────────────
EXISTING INFRASTRUCTURE (no capital cost):
  Grid:    Brayton η = 38.0%
  Furnace: AFUE = 95%, fuel = $0.012/kWh
  Heat Pump: COP = f(ΔT), existing installation
──────────────────────────────────────────────────
NEW INVESTMENTS (capacity to be optimized):
  Solar PV:      @ $1000/kW  (rooftop limited)
  Wind Turbine:  0-50 kW @ $1500/kW
  Battery:       0-200 kWh @ $300/kWh
  Solar Thermal: @ $200/m²  (rooftop limited)
──────────────────────────────────────────────────
CONSTRAINTS:
  Rooftop Area: PV + Solar Thermal ≤ 60.0 m²
  Wind Swept Area (for exergy): 50.0 m² (D = 8.0 m)
──────────────────────────────────────────────────
EMISSIONS FACTORS:
  Natural Gas:      0.181 kg CO₂/kWh_fuel
  Grid Electricity: 0.452 kg CO₂/kWh_elec (η=40%)


## Optimization Model

### Decision Variables

**Capacity Variables (to be optimized):**
- $A_{pv}$: PV panel area (m²)
- $P_{wt,rated}$: Wind turbine rated power (W)
- $E_{bat}$: Battery energy capacity (Wh)
- $A_{stc}$: Solar thermal collector area (m²)

**Dispatch Variables (hourly):**
- $P_{grid}(t)$: Grid electricity import (W)
- $P_{pv}(t)$: PV power output (W)
- $P_{wt}(t)$: Wind power output (W)
- $P_{ch}(t), P_{dch}(t)$: Battery charge/discharge (W)
- $Q_{furnace}(t)$: Furnace heat output (W) - **existing, no capital**
- $Q_{hp,h}(t), Q_{hp,c}(t)$: Heat pump output (W) - **existing, no capital**
- $Q_{stc}(t)$: Solar thermal output (W)

**Note:** Grid and Heat Pump/Furnace are existing infrastructure - only operating costs apply.

In [127]:
# Create optimization model
model = Model(HiGHS.Optimizer)
set_silent(model)

# Time index
t_set = 1:T

# ============================================================
# CAPACITY DECISION VARIABLES (to be optimized)
# ============================================================
@variable(model, 0 <= Cap_pv <= A_roof_max)    # PV area [m²] - rooftop limited
@variable(model, 0 <= Cap_wt <= 50000)          # Wind rated power [W]
@variable(model, 0 <= Cap_bat <= 200000)        # Battery capacity [Wh]
@variable(model, 0 <= Cap_stc <= A_roof_max)   # Solar thermal area [m²] - rooftop limited
@variable(model, 0 <= Cap_pcm <= 200000)        # PCM thermal storage capacity [Wh_th]

# ============================================================
# ROOFTOP AREA CONSTRAINT
# ============================================================
@constraint(model, rooftop_area, Cap_pv + Cap_stc <= A_roof_max)

# ============================================================
# DISPATCH DECISION VARIABLES (hourly)
# ============================================================
@variable(model, P_grid[t_set] >= 0)       # Grid import [W] - NO CAPITAL
@variable(model, P_pv[t_set] >= 0)         # PV output [W]
@variable(model, P_wt[t_set] >= 0)         # Wind output [W]
@variable(model, P_ch[t_set] >= 0)         # Battery charge [W]
@variable(model, P_dch[t_set] >= 0)        # Battery discharge [W]
@variable(model, Q_furnace[t_set] >= 0)    # Furnace heat [W] - NO CAPITAL (existing)
@variable(model, Q_hp_h[t_set] >= 0)       # HP heating [W] - NO CAPITAL (existing)
@variable(model, Q_hp_c[t_set] >= 0)       # HP cooling [W] - NO CAPITAL (existing)
@variable(model, Q_stc[t_set] >= 0)        # Solar thermal [W]
@variable(model, Q_pcm_ch[t_set] >= 0)     # PCM charge [W]
@variable(model, Q_pcm_dch[t_set] >= 0)    # PCM discharge [W]

# State variables
@variable(model, SOC[t_set] >= 0)          # Battery SOC [Wh] - upper bound depends on Cap_bat
@variable(model, E_pcm[t_set] >= 0)        # PCM energy [Wh] - upper bound depends on Cap_pcm

println("Decision variables defined")
println("  Capacity variables: Cap_pv, Cap_wt, Cap_bat, Cap_stc, Cap_pcm")
println("  Rooftop constraint: Cap_pv + Cap_stc ≤ $(A_roof_max) m²")
println("  Dispatch variables: $(num_variables(model) - 5) time-indexed variables")

Decision variables defined
  Capacity variables: Cap_pv, Cap_wt, Cap_bat, Cap_stc, Cap_pcm
  Rooftop constraint: Cap_pv + Cap_stc ≤ 60.0 m²
  Dispatch variables: 113880 time-indexed variables


### Constraints

In [128]:
# ============================================================
# EXTRACT DATA VECTORS
# ============================================================
E_demand = data.electricity_demand * 1000  # W
Q_heat_demand = data.heating_demand * 1000  # W
Q_cool_demand = data.cooling_demand * 1000  # W
v_wind = data.wind_speed  # m/s
G_solar = data.solar_irradiance  # W/m²
T_amb = data.temperature .+ 273.15  # K

# Battery parameters (constants, not from struct)
η_rt_bat = 0.86        # Round-trip efficiency
η_ch = sqrt(η_rt_bat)  # Charge efficiency = 0.927
η_dch = sqrt(η_rt_bat) # Discharge efficiency = 0.927

println("Data loaded:")
println("  Time steps: $T hours")
println("  Peak electricity demand: $(round(maximum(E_demand)/1000, digits=1)) kW")
println("  Total heating demand: $(round(sum(Q_heat_demand)/1000, digits=1)) kWh")
println("  Total cooling demand: $(round(sum(Q_cool_demand)/1000, digits=1)) kWh")

Data loaded:
  Time steps: 8760 hours
  Peak electricity demand: 1.7 kW
  Total heating demand: 14804.1 kWh
  Total cooling demand: 12529.3 kWh


In [129]:
# ============================================================
# RENEWABLE GENERATION LIMITS (linked to capacity variables)
# ============================================================

# Pre-calculate efficiency/capacity factors for each hour
η_pv_t = zeros(T)      # PV efficiency at each hour
CF_wt_t = zeros(T)     # Wind capacity factor at each hour  
η_stc_t = zeros(T)     # Solar thermal efficiency at each hour

for t in t_set
    G = G_solar[t]
    v = v_wind[t]
    
    # PV: efficiency depends on temperature (use reference PV for calculation)
    if G > 0
        T_cell = (T_amb[t] - 273.15) + ((45 - 20) / 800) * G + 273.15
        η_pv_t[t] = 0.20 * (1 - 0.004 * (T_cell - 298.15))
        η_pv_t[t] = max(η_pv_t[t], 0.0)
    end
    
    # Wind: capacity factor based on wind speed (simplified power curve)
    ρ_air = 1.225
    C_p = 0.45
    v_cutin, v_rated, v_cutout = 3.0, 12.0, 25.0
    
    if v < v_cutin || v > v_cutout
        CF_wt_t[t] = 0.0
    elseif v >= v_rated
        CF_wt_t[t] = 1.0
    else
        CF_wt_t[t] = min(1.0, (v / v_rated)^3)
    end
    
    # Solar thermal: useful heat per m² (simplified HWB equation)
    if G > 0
        F_R, τα, U_L = 0.80, 0.82, 4.5
        T_inlet = 323.15  # 50°C
        q_useful = F_R * (G * τα - U_L * (T_inlet - T_amb[t]))
        η_stc_t[t] = G > 0 ? max(q_useful / G, 0.0) : 0.0  # Julia ternary syntax
    end
end

# Add constraints linking dispatch to capacity
for t in t_set
    # PV: P_pv(t) ≤ η(t) × G(t) × Cap_pv
    @constraint(model, P_pv[t] <= η_pv_t[t] * G_solar[t] * Cap_pv)
    
    # Wind: P_wt(t) ≤ CF(t) × Cap_wt
    @constraint(model, P_wt[t] <= CF_wt_t[t] * Cap_wt)
    
    # Solar thermal: Q_stc(t) ≤ η(t) × G(t) × Cap_stc
    @constraint(model, Q_stc[t] <= η_stc_t[t] * G_solar[t] * Cap_stc)
end

println("Renewable generation limits added (linked to capacity variables)")

Renewable generation limits added (linked to capacity variables)


In [130]:
# ============================================================
# EQUIPMENT CAPACITY CONSTRAINTS
# ============================================================

# Existing equipment (fixed capacity, no capital cost)
Q_furnace_max = 20000.0   # 20 kW existing furnace
Q_hp_max = 15000.0        # 15 kW existing heat pump

# Battery constraints (linked to capacity variable)
C_rate = 0.5  # Max charge/discharge rate = 0.5C (2 hour full charge)

for t in t_set
    # Battery power limits depend on capacity
    @constraint(model, P_ch[t] <= C_rate * Cap_bat)
    @constraint(model, P_dch[t] <= C_rate * Cap_bat)
    
    # SOC bounded by capacity
    @constraint(model, SOC[t] <= Cap_bat)
    
    # Existing equipment (fixed limits, NO CAPITAL COST)
    @constraint(model, Q_furnace[t] <= Q_furnace_max)
    @constraint(model, Q_hp_h[t] <= Q_hp_max)
    @constraint(model, Q_hp_c[t] <= Q_hp_max)
end

println("Equipment capacity constraints added")
println("  Battery: charge/discharge ≤ $(C_rate)C × Cap_bat")
println("  Furnace: existing $(Q_furnace_max/1000) kW (no capital)")
println("  Heat Pump: existing $(Q_hp_max/1000) kW (no capital)")

Equipment capacity constraints added
  Battery: charge/discharge ≤ 0.5C × Cap_bat
  Furnace: existing 20.0 kW (no capital)
  Heat Pump: existing 15.0 kW (no capital)


In [131]:
# Electricity balance
for t in t_set
    # COP calculations for HP electrical consumption
    T_outdoor = T_amb[t]
    ΔT_h = T_indoor_set - T_outdoor  # Heating lift
    ΔT_c = T_indoor_set - T_outdoor  # Cooling lift (simplified)
    
    COP_h = max(1.0, 6.08 - 0.0941*abs(ΔT_h) + 0.000464*ΔT_h^2)
    COP_c = max(1.0, 5.08 - 0.0941*abs(ΔT_c) + 0.000464*ΔT_c^2)
    
    # Electricity balance: supply = demand
    @constraint(model, 
        P_grid[t] + P_pv[t] + P_wt[t] + P_dch[t] == 
        E_demand[t] + P_ch[t] + Q_hp_h[t]/COP_h + Q_hp_c[t]/COP_c
    )
end

println("Electricity balance constraints added")

Electricity balance constraints added


In [132]:
# Heating balance
for t in t_set
    @constraint(model,
        Q_furnace[t] + Q_hp_h[t] + Q_stc[t] + Q_pcm_dch[t] >= 
        Q_heat_demand[t] + Q_pcm_ch[t]
    )
end

# Cooling balance
for t in t_set
    @constraint(model, Q_hp_c[t] >= Q_cool_demand[t])
end

println("Thermal balance constraints added")

Thermal balance constraints added


In [133]:
# ============================================================
# STORAGE DYNAMICS
# ============================================================

# Battery efficiency
η_bat = 0.93  # One-way efficiency (round-trip = 0.86)

# Battery SOC dynamics (initial SOC = 50% of optimized capacity)
@constraint(model, SOC[1] == 0.5 * Cap_bat + (η_bat * P_ch[1] - P_dch[1]/η_bat) * Δt)

for t in 2:T
    @constraint(model, SOC[t] == SOC[t-1] + (η_bat * P_ch[t] - P_dch[t]/η_bat) * Δt)
end

# ============================================================
# PCM STORAGE DYNAMICS (capacity is now a decision variable)
# ============================================================
# PCM efficiency
η_pcm_ch = 0.90   # Charging efficiency (heat loss during charging)
η_pcm_dch = 0.95  # Discharging efficiency

# PCM charge/discharge rate limits (0.5C rate = 2 hour full charge)
C_rate_pcm = 0.5

for t in t_set
    # PCM power limits depend on capacity
    @constraint(model, Q_pcm_ch[t] <= C_rate_pcm * Cap_pcm)
    @constraint(model, Q_pcm_dch[t] <= C_rate_pcm * Cap_pcm)
    
    # PCM energy bounded by capacity
    @constraint(model, E_pcm[t] <= Cap_pcm)
end

# PCM energy dynamics (initial = 50% of optimized capacity)
@constraint(model, E_pcm[1] == 0.5 * Cap_pcm + (η_pcm_ch * Q_pcm_ch[1] - Q_pcm_dch[1]/η_pcm_dch) * Δt)

for t in 2:T
    @constraint(model, E_pcm[t] == E_pcm[t-1] + (η_pcm_ch * Q_pcm_ch[t] - Q_pcm_dch[t]/η_pcm_dch) * Δt)
end

println("Storage dynamics constraints added")
println("  Battery: η_ch = η_dch = $(η_bat) (round-trip = $(η_bat^2))")
println("  PCM: η_ch = $(η_pcm_ch), η_dch = $(η_pcm_dch)")

Storage dynamics constraints added
  Battery: η_ch = η_dch = 0.93 (round-trip = 0.8649000000000001)
  PCM: η_ch = 0.9, η_dch = 0.95


### CO₂ Emission Constraint

To achieve decarbonization targets, we add a constraint limiting total annual CO₂ emissions:

$$\sum_t \left[ P_{grid}(t) \cdot EF_{grid} + \frac{Q_{furnace}(t)}{\eta_{AFUE}} \cdot EF_{NG} \right] \cdot \Delta t \leq CO_2^{max}$$

Where:
- $EF_{grid} = 0.452$ kg CO₂/kWh (grid electricity from NG power plant)
- $EF_{NG} = 0.181$ kg CO₂/kWh (natural gas combustion)
- $CO_2^{max}$ = Maximum allowed annual emissions (kg)

In [134]:
# ============================================================
# CO₂ EMISSION CONSTRAINT
# ============================================================

# Maximum allowed annual emissions (kg CO₂)
# Current baseline: ~5246 kg CO₂/year (100% grid + no renewables)
# Set target below baseline to force renewable investment

CO2_max = 1738.0  # kg CO₂ - 

# Total emissions expression (kg CO₂)
# Grid: P_grid [W] × Δt [h] / 1000 [W→kW] × EF_grid [kg/kWh]
# Furnace: Q_furnace [W] / η_furnace × Δt [h] / 1000 × EF_NG [kg/kWh_fuel]

@expression(model, total_emissions,
    sum(P_grid[t] * Δt / 1000 * EF_grid for t in t_set) +           # Grid electricity emissions
    sum(Q_furnace[t] / 0.95 * Δt / 1000 * EF_NG for t in t_set) # Natural gas combustion emissions
)

# Add emission constraint
@constraint(model, emission_limit, total_emissions <= CO2_max)

println("="^60)
println("         CO₂ EMISSION CONSTRAINT ADDED")
println("="^60)
@printf("Maximum allowed emissions: %.0f kg CO₂/year\n", CO2_max)
@printf("Baseline emissions (no constraint): ~5246 kg CO₂/year\n")
@printf("Required reduction: %.1f%%\n", (1 - CO2_max/5246) * 100)
println("="^60)

         CO₂ EMISSION CONSTRAINT ADDED
Maximum allowed emissions: 1738 kg CO₂/year
Baseline emissions (no constraint): ~5246 kg CO₂/year
Required reduction: 66.9%


### Objective Function

**Minimize: Operating Cost + Capital Cost of NEW Equipment**

$$\min F = \underbrace{\sum_t \left[ c_{grid} \cdot P_{grid}(t) + c_{NG} \cdot \frac{Q_{furnace}(t)}{\eta_{AFUE}} \right] \cdot \Delta t}_{\text{Operating Cost}} + \underbrace{\sum_k \dot{Z}_k \cdot T_{sim}}_{\text{Capital Cost (new equipment only)}}$$

**Existing infrastructure (NO capital cost):**
- Grid electricity (Brayton cycle power plant)
- Gas furnace
- Heat pump

**New investments (capital cost optimized):**
- Solar PV: $1000/kW × (Cap_pv × 0.20 kW/m²)
- Wind turbine: $1500/kW × Cap_wt
- Battery: $300/kWh × Cap_bat
- Solar thermal: $200/m² × Cap_stc

In [135]:
# ============================================================
# OBJECTIVE FUNCTION
# ============================================================

# Operating cost coefficients
c_grid = 0.45 / 1000   # $/Wh for grid electricity
c_ng = 0.13 / 1000    # $/Wh for natural gas
η_furnace = 0.95       # Furnace AFUE

# ============================================================
# OPERATING COST (grid + gas)
# ============================================================
@expression(model, cost_operating, 
    sum(c_grid * P_grid[t] * Δt for t in t_set) +           # Grid electricity
    sum(c_ng * Q_furnace[t] / η_furnace * Δt for t in t_set) # Natural gas
)

# ============================================================
# CAPITAL COST (only NEW investments)
# ============================================================
# Unit costs
cost_pv_per_kW   = 2900.0    # $/kW   (Residential PV installed cost ≈ 2.90 $/Wdc, NREL benchmark)
cost_wt_per_kW   = 7370.0    # $/kW   (Small wind installed cost, 2023 avg, PNNL distributed wind report)
cost_bat_per_kWh = 1152.0    # $/kWh  (Derived from NREL PV-only vs PV+storage delta; system-level)
cost_stc_per_m2  = 1000.0    # $/m²   (Residential solar thermal / solar water heater order-of-magnitude, DOE)
cost_pcm_per_kWh = 30.0      # $/kWh_th (Elfeky et al., Energy 2022 - LHS with PCM cost range)

# Lifetimes
life_pv = 25
life_wt = 20
life_bat = 10
life_stc = 25
life_pcm = 20

# Annualized capital cost rate ($/h) for each technology
# Cap_pv is in m², need to convert to kW: kW = m² × η_ref × 1 kW/m² at STC
# = m² × 0.20 × 1 = 0.20 kW per m²
ann_pv = cost_pv_per_kW * 0.20 * CRF(life_pv) / hours_per_year  # $ per m² per hour
ann_wt = cost_wt_per_kW * CRF(life_wt) / hours_per_year / 1000  # $ per W per hour
ann_bat = cost_bat_per_kWh * CRF(life_bat) / hours_per_year / 1000  # $ per Wh per hour
ann_stc = cost_stc_per_m2 * CRF(life_stc) / hours_per_year  # $ per m² per hour
ann_pcm = cost_pcm_per_kWh * CRF(life_pcm) / hours_per_year / 1000  # $ per Wh_th per hour

# Capital cost expression (linear in capacity variables)
@expression(model, cost_capital,
    ann_pv * Cap_pv * T +        # PV capital
    ann_wt * Cap_wt * T +        # Wind capital
    ann_bat * Cap_bat * T +      # Battery capital
    ann_stc * Cap_stc * T +      # Solar thermal capital
    ann_pcm * Cap_pcm * T        # PCM thermal storage capital
)

# NO capital cost for: Grid, Furnace, Heat Pump (existing!)

# ============================================================
# TOTAL OBJECTIVE
# ============================================================
@objective(model, Min, cost_operating + cost_capital)

println("Objective function set")
println("─"^50)
println("OPERATING COSTS:")
println("  Grid electricity: \$$(c_grid*1000)/kWh")
println("  Natural gas:      \$$(c_ng*1000)/kWh_fuel")
println("─"^50)
println("CAPITAL COSTS (new investments only):")
@printf("  PV:            \$%.4f per m² per hour\n", ann_pv)
@printf("  Wind:          \$%.6f per W per hour\n", ann_wt)
@printf("  Battery:       \$%.6f per Wh per hour\n", ann_bat)
@printf("  Solar thermal: \$%.4f per m² per hour\n", ann_stc)
@printf("  PCM storage:   \$%.6f per Wh_th per hour\n", ann_pcm)
println("─"^50)
println("NO CAPITAL for: Grid, Furnace, Heat Pump (existing)")

Objective function set
──────────────────────────────────────────────────
OPERATING COSTS:
  Grid electricity: $0.45/kWh
  Natural gas:      $0.13/kWh_fuel
──────────────────────────────────────────────────
CAPITAL COSTS (new investments only):
  PV:            $0.0062 per m² per hour
  Wind:          $0.000086 per W per hour
  Battery:       $0.000020 per Wh per hour
  Solar thermal: $0.0107 per m² per hour
  PCM storage:   $0.000000 per Wh_th per hour
──────────────────────────────────────────────────
NO CAPITAL for: Grid, Furnace, Heat Pump (existing)


## Solve Optimization

In [136]:
optimize!(model)

println("\n=== Optimization Results ===")
println("Status: ", termination_status(model))
println("Objective value: \$", round(objective_value(model), digits=2))


=== Optimization Results ===
Status: OPTIMAL
Objective value: $5587.81


## Results Analysis

In [137]:
# ============================================================
# EXTRACT OPTIMAL VALUES
# ============================================================

# Optimal capacities (NEW INVESTMENTS)
Cap_pv_opt = value(Cap_pv)
Cap_wt_opt = value(Cap_wt)
Cap_bat_opt = value(Cap_bat)
Cap_stc_opt = value(Cap_stc)
Cap_pcm_opt = value(Cap_pcm)

# Optimal dispatch
P_grid_opt = value.(P_grid)
P_pv_opt = value.(P_pv)
P_wt_opt = value.(P_wt)
P_ch_opt = value.(P_ch)
P_dch_opt = value.(P_dch)
Q_furnace_opt = value.(Q_furnace)
Q_hp_h_opt = value.(Q_hp_h)
Q_hp_c_opt = value.(Q_hp_c)
Q_stc_opt = value.(Q_stc)
Q_pcm_ch_opt = value.(Q_pcm_ch)
Q_pcm_dch_opt = value.(Q_pcm_dch)
SOC_opt = value.(SOC)
E_pcm_opt = value.(E_pcm)

# ============================================================
# OPTIMAL CAPACITY RESULTS
# ============================================================
println("\n" * "="^60)
println("         OPTIMAL CAPACITIES (New Investments)")
println("="^60)
capacity_results = DataFrame(
    Technology = ["Solar PV", "Wind Turbine", "Battery", "Solar Thermal", "PCM Storage"],
    Optimal_Size = [Cap_pv_opt, Cap_wt_opt/1000, Cap_bat_opt/1000, Cap_stc_opt, Cap_pcm_opt/1000],
    Unit = ["m²", "kW", "kWh", "m²", "kWh_th"],
    Capital_Cost = [
        Cap_pv_opt * 0.20 * cost_pv_per_kW,
        Cap_wt_opt/1000 * cost_wt_per_kW,
        Cap_bat_opt/1000 * cost_bat_per_kWh,
        Cap_stc_opt * cost_stc_per_m2,
        Cap_pcm_opt/1000 * cost_pcm_per_kWh
    ]
)
println(capacity_results)
@printf("\nTotal Investment: \$%.2f\n", sum(capacity_results.Capital_Cost))

# ============================================================
# ENERGY SUMMARY
# ============================================================
println("\n" * "="^60)
println("              ENERGY SUMMARY ($(T) hours)")
println("="^60)
energy_results = DataFrame(
    Metric = [
        "Grid Import (existing)",
        "PV Generation (new)",
        "Wind Generation (new)",
        "Furnace Heat (existing)",
        "HP Heating (existing)",
        "HP Cooling (existing)",
        "Solar Thermal (new)",
        "PCM Discharge (new)",
        "Battery Throughput"
    ],
    Value = [
        sum(P_grid_opt) * Δt / 1000,
        sum(P_pv_opt) * Δt / 1000,
        sum(P_wt_opt) * Δt / 1000,
        sum(Q_furnace_opt) * Δt / 1000,
        sum(Q_hp_h_opt) * Δt / 1000,
        sum(Q_hp_c_opt) * Δt / 1000,
        sum(Q_stc_opt) * Δt / 1000,
        sum(Q_pcm_dch_opt) * Δt / 1000,
        sum(P_ch_opt) * Δt / 1000
    ],
    Unit = ["kWh", "kWh", "kWh", "kWh_th", "kWh_th", "kWh_th", "kWh_th", "kWh_th", "kWh"]
)
println(energy_results)


         OPTIMAL CAPACITIES (New Investments)
5×4 DataFrame
 Row │ Technology     Optimal_Size  Unit    Capital_Cost 
     │ String         Float64       String  Float64      
─────┼───────────────────────────────────────────────────
   1 │ Solar PV            35.9067  m²          20825.9
   2 │ Wind Turbine         0.0     kW              0.0
   3 │ Battery             10.0993  kWh         11634.4
   4 │ Solar Thermal       -0.0     m²             -0.0
   5 │ PCM Storage         57.1439  kWh_th       1714.32

Total Investment: $34174.56

              ENERGY SUMMARY (8760 hours)
9×3 DataFrame
 Row │ Metric                   Value     Unit   
     │ String                   Float64   String 
─────┼───────────────────────────────────────────
   1 │ Grid Import (existing)    3840.88  kWh
   2 │ PV Generation (new)       8338.08  kWh
   3 │ Wind Generation (new)        0.0   kWh
   4 │ Furnace Heat (existing)      0.0   kWh_th
   5 │ HP Heating (existing)    15967.8   kWh_th
   6 │ HP Co

## Exergy Balance, Efficiency, and Emissions Analysis

The system boundary includes the **power plant** and **fixed resource areas** (rooftop for solar, swept area for wind), providing a complete thermodynamic and environmental picture.

```
┌──────────────────────────────────────────────────────────────────────────┐
│                        EXTENDED SYSTEM BOUNDARY                          │
│                                                                          │
│   Fuel Exergy ──►┌─────────────┐                                        │
│   (NG at plant)  │ Power Plant │──► Grid Electricity                    │
│                  │ (Brayton)   │         │                              │
│                  └─────────────┘         ▼                              │
│                                    ┌─────────────┐    ┌─────────────┐   │
│   Solar Exergy ─────────────────► │  Building   │───►│   Useful    │   │
│   (Full rooftop = 60 m²)          │  (PV, STC)  │    │   Output    │   │
│         ↓ (uncovered area)         └─────────────┘    └─────────────┘   │
│      Passes through as LOSS                                              │
│                                                                          │
│   Wind Exergy ────────────────────────────┘                             │
│   (Fixed A = 50 m²)         ↓ (if not captured)                         │
│                        Passes through as LOSS                            │
└──────────────────────────────────────────────────────────────────────────┘
```

### Consistent Treatment of Renewable Resources

| Resource | Input Basis | If Captured | If NOT Captured |
|----------|-------------|-------------|-----------------|
| **Solar** | Full rooftop (60 m²) | Destruction on panel area | **Loss** (passes through) |
| **Wind** | Fixed swept area (50 m²) | Destruction in turbine | **Loss** (passes through) |

### Exergy Balance Equation
$$\dot{Ex}_{in} = \dot{Ex}_{out,useful} + \dot{Ex}_{D} + \dot{Ex}_{loss}$$

Where:
- $\dot{Ex}_{D}$ = Destruction (internal irreversibility in equipment)
- $\dot{Ex}_{loss}$ = Uncaptured resources (solar/wind passes through)

### Exergy Inputs:
1. **Fuel at Power Plant**: $\dot{Ex}_{fuel} = \frac{P_{grid}}{\eta_{plant}} \cdot \psi_{NG}$ where $\psi_{NG} = 1.06$
2. **Natural Gas Direct**: $\dot{Ex}_{NG} = \psi_{NG} \cdot Q_{fuel}$
3. **Solar Radiation**: $\dot{Ex}_{solar} = \psi_{solar} \cdot G \cdot A_{roof}$ where $\psi_{solar} \approx 0.93$, $A_{roof} = 60$ m²
4. **Wind Kinetic**: $\dot{Ex}_{wind} = \frac{1}{2}\rho A_{swept} v^3$ where $A_{swept} = 50$ m²

### CO₂ Emissions:
$$\dot{m}_{CO_2} = E_{grid} \cdot EF_{grid} + E_{fuel,furnace} \cdot EF_{NG}$$

Where:
- $EF_{NG} = 0.181$ kg CO₂/kWh (natural gas combustion)
- $EF_{grid} = EF_{NG} / \eta_{plant}$ kg CO₂/kWh (accounts for power plant efficiency)

In [138]:
# ============================================================
# COMPREHENSIVE EXERGY BALANCE
# ============================================================
# System Boundary: Extended to include power plant (Option B)
# This captures the full thermodynamic picture

# Exergy factors (using local variables to avoid conflicts)
ψ_solar_ex = 0.93       # Petela-Landsberg-Press solar exergy factor
T_sun_ex = 5778.0       # Sun surface temperature [K]
T_heat_delivery = 323.15  # Heating delivery temperature [K] (50°C)
T_cool_delivery = 280.15  # Cooling delivery temperature [K] (7°C)
ρ_air = 1.225           # Air density [kg/m³]

# ============================================================
# 1. EXERGY INPUTS (Extended System Boundary)
# ============================================================

# Grid: Account for FUEL exergy at power plant (not just electricity)
# Fuel input = Electricity / η_plant
# Fuel exergy = Fuel input × ψ_NG (chemical exergy factor)
Ex_in_grid_fuel = sum(P_grid_opt) / η_powerplant * ψ_NG * Δt / 1000  # kWh_ex

# Natural gas chemical exergy (to furnace) - direct use at building
Q_fuel_furnace = sum(Q_furnace_opt) / η_furnace  # Fuel input [W]
Ex_in_NG = Q_fuel_furnace * ψ_NG * Δt / 1000  # kWh_ex

# ============================================================
# Solar radiation exergy - FULL ROOFTOP (consistent with wind)
# Solar falls on entire rooftop whether panels installed or not
# ============================================================
Ex_in_solar = 0.0
for t in t_set
    G = G_solar[t]
    if G > 0
        # Exergy from solar on ENTIRE rooftop area (not just panels)
        Ex_in_solar += ψ_solar_ex * G * A_roof_max * Δt / 1000  # kWh_ex
    end
end

# ============================================================
# Wind kinetic exergy - FIXED SWEPT AREA
# Wind flows through swept area whether turbine installed or not
# ============================================================
Ex_in_wind = 0.0
for t in t_set
    v = v_wind[t]
    if v > 0
        # Kinetic power = 0.5 × ρ × A × v³
        P_wind_available = 0.5 * ρ_air * A_swept_max * v^3  # W
        Ex_in_wind += P_wind_available * Δt / 1000  # kWh (kinetic energy = exergy)
    end
end

# Total exergy input (extended boundary)
Ex_in_total = Ex_in_grid_fuel + Ex_in_NG + Ex_in_solar + Ex_in_wind

println("\n" * "="^70)
println("            EXERGY BALANCE ANALYSIS (Extended Boundary)")
println("="^70)
println("System boundary includes: Power plant → Building → Useful output")
println("Solar resource: Full rooftop = $(A_roof_max) m² (panels use $(round(Cap_pv_opt + Cap_stc_opt, digits=1)) m²)")
println("Wind resource: Fixed swept area = $(A_swept_max) m² (D = $(round(D_rotor, digits=1)) m)")

println("\n┌─────────────────────────────────────────────────────────────────────┐")
println("│                        EXERGY INPUTS                                │")
println("├─────────────────────────────────────────────────────────────────────┤")
@printf("│  Grid Fuel at Power Plant (ψ=1.06)  : %10.2f kWh_ex             │\n", Ex_in_grid_fuel)
@printf("│  Natural Gas Direct (ψ=1.06)        : %10.2f kWh_ex             │\n", Ex_in_NG)
@printf("│  Solar Radiation (ψ=0.93, A=%.0f m²) : %10.2f kWh_ex             │\n", A_roof_max, Ex_in_solar)
@printf("│  Wind Kinetic (A=%.0f m²)            : %10.2f kWh_ex             │\n", A_swept_max, Ex_in_wind)
println("├─────────────────────────────────────────────────────────────────────┤")
@printf("│  TOTAL EXERGY INPUT                 : %10.2f kWh_ex             │\n", Ex_in_total)
println("└─────────────────────────────────────────────────────────────────────┘")


            EXERGY BALANCE ANALYSIS (Extended Boundary)
System boundary includes: Power plant → Building → Useful output
Solar resource: Full rooftop = 60.0 m² (panels use 35.9 m²)
Wind resource: Fixed swept area = 50.0 m² (D = 8.0 m)

┌─────────────────────────────────────────────────────────────────────┐
│                        EXERGY INPUTS                                │
├─────────────────────────────────────────────────────────────────────┤
│  Grid Fuel at Power Plant (ψ=1.06)  :   10178.34 kWh_ex             │
│  Natural Gas Direct (ψ=1.06)        :       0.00 kWh_ex             │
│  Solar Radiation (ψ=0.93, A=60 m²) :  107205.25 kWh_ex             │
│  Wind Kinetic (A=50 m²)            :   17242.00 kWh_ex             │
├─────────────────────────────────────────────────────────────────────┤
│  TOTAL EXERGY INPUT                 :  134625.59 kWh_ex             │
└─────────────────────────────────────────────────────────────────────┘


In [139]:
# ============================================================
# 2. EXERGY OUTPUTS (Useful Product)
# ============================================================

# Electrical demand satisfied (electricity = pure exergy)
Ex_out_elec = sum(E_demand) * Δt / 1000  # kWh_ex

# Heating exergy delivered (Carnot factor)
carnot_heat = 1 - T_ref / T_heat_delivery  # ~0.077 for 50°C heating
Ex_out_heat = 0.0
for t in t_set
    Q_heat_t = Q_furnace_opt[t] + Q_hp_h_opt[t] + Q_stc_opt[t]
    Ex_out_heat += Q_heat_t * carnot_heat * Δt / 1000  # kWh_ex
end

# Cooling exergy delivered (reverse Carnot)
carnot_cool = T_ref / T_cool_delivery - 1  # ~0.064 for 7°C cooling
Ex_out_cool = 0.0
for t in t_set
    Ex_out_cool += Q_hp_c_opt[t] * carnot_cool * Δt / 1000  # kWh_ex
end

# Total useful exergy output
Ex_out_total = Ex_out_elec + Ex_out_heat + Ex_out_cool

println("\n┌─────────────────────────────────────────────────────────────────────┐")
println("│                     EXERGY OUTPUTS (Useful)                         │")
println("├─────────────────────────────────────────────────────────────────────┤")
@printf("│  Electrical Load (pure exergy)     : %10.2f kWh_ex             │\n", Ex_out_elec)
@printf("│  Heating Exergy (Carnot=%.3f)     : %10.2f kWh_ex             │\n", carnot_heat, Ex_out_heat)
@printf("│  Cooling Exergy (Carnot=%.3f)     : %10.2f kWh_ex             │\n", carnot_cool, Ex_out_cool)
println("├─────────────────────────────────────────────────────────────────────┤")
@printf("│  TOTAL USEFUL EXERGY OUTPUT        : %10.2f kWh_ex             │\n", Ex_out_total)
println("└─────────────────────────────────────────────────────────────────────┘")


┌─────────────────────────────────────────────────────────────────────┐
│                     EXERGY OUTPUTS (Useful)                         │
├─────────────────────────────────────────────────────────────────────┤
│  Electrical Load (pure exergy)     :    5738.15 kWh_ex             │
│  Heating Exergy (Carnot=0.077)     :    1235.32 kWh_ex             │
│  Cooling Exergy (Carnot=0.064)     :     805.13 kWh_ex             │
├─────────────────────────────────────────────────────────────────────┤
│  TOTAL USEFUL EXERGY OUTPUT        :    7778.60 kWh_ex             │
└─────────────────────────────────────────────────────────────────────┘


In [140]:
# ============================================================
# 3. EXERGY DESTRUCTION BY COMPONENT
# ============================================================

# Grid (Brayton cycle power plant) - destruction happens at power plant
# For imported electricity, the fuel exergy consumed at plant was:
# Ex_fuel_at_plant = P_grid / η_plant × ψ_NG, destruction = Ex_fuel - P_grid
Ex_D_grid_plant = sum(P_grid_opt) * (ψ_NG/η_powerplant - 1) * Δt / 1000  # kWh_ex

# ============================================================
# SOLAR PV - Destruction only on INSTALLED panel area
# Loss = solar on uncovered rooftop (passes through)
# ============================================================
Ex_D_pv = 0.0
Ex_loss_solar_pv = 0.0  # Solar on rooftop not covered by PV

for t in t_set
    G = G_solar[t]
    if G > 0
        # Solar exergy on PV panel area → some converted, rest is destruction
        Ex_solar_on_pv_t = ψ_solar_ex * G * Cap_pv_opt * Δt / 1000
        Ex_elec_out_t = P_pv_opt[t] * Δt / 1000
        Ex_D_pv += max(0, Ex_solar_on_pv_t - Ex_elec_out_t)
        
        # Solar exergy on uncovered rooftop (not PV, not STC) → passes through as LOSS
        A_uncovered = A_roof_max - Cap_pv_opt - Cap_stc_opt
        Ex_loss_solar_pv += ψ_solar_ex * G * max(0, A_uncovered) * Δt / 1000
    end
end

# ============================================================
# SOLAR THERMAL - Destruction only on INSTALLED collector area
# ============================================================
Ex_D_stc = 0.0
for t in t_set
    G = G_solar[t]
    if G > 0
        Ex_solar_on_stc_t = ψ_solar_ex * G * Cap_stc_opt * Δt / 1000
        Ex_heat_out_t = Q_stc_opt[t] * carnot_heat * Δt / 1000
        Ex_D_stc += max(0, Ex_solar_on_stc_t - Ex_heat_out_t)
    end
end

# ============================================================
# WIND TURBINE - Destruction only if turbine installed
# Loss = wind through swept area not captured (passes through)
# ============================================================
Ex_captured_wind = sum(P_wt_opt) * Δt / 1000  # Electricity captured from wind
Ex_D_wt = 0.0      # Destruction in turbine (conversion losses)
Ex_loss_wind = 0.0  # Uncaptured wind (passes through)

if Cap_wt_opt > 0
    # Wind turbine installed - some is captured, rest is destruction
    for t in t_set
        v = v_wind[t]
        if v > 0
            P_wind_available = 0.5 * ρ_air * A_swept_max * v^3  # W
            Ex_available_t = P_wind_available * Δt / 1000
            Ex_captured_t = P_wt_opt[t] * Δt / 1000
            Ex_D_wt += max(0, Ex_available_t - Ex_captured_t)
        end
    end
else
    # No turbine - all wind passes through uncaptured
    Ex_loss_wind = Ex_in_wind
end

# Battery - destruction from round-trip losses
Ex_D_battery = sum(P_ch_opt) * (1 - η_rt_bat) * Δt / 1000  # kWh_ex

# Furnace - destruction = chemical exergy - useful heat exergy
Ex_D_furnace = 0.0
for t in t_set
    Q_fuel_t = Q_furnace_opt[t] / η_furnace
    Ex_fuel_t = Q_fuel_t * ψ_NG * Δt / 1000
    Ex_heat_useful_t = Q_furnace_opt[t] * carnot_heat * Δt / 1000
    Ex_D_furnace += max(0, Ex_fuel_t - Ex_heat_useful_t)
end

# Heat Pump - destruction (electricity in - thermal exergy out)
Ex_D_hp = 0.0
for t in t_set
    T_outdoor = T_amb[t]
    ΔT_h = T_indoor_set - T_outdoor
    COP_h = max(1.0, 6.08 - 0.0941*abs(ΔT_h) + 0.000464*ΔT_h^2)
    
    W_hp_t = Q_hp_h_opt[t] / COP_h  # Electricity consumed
    Ex_elec_in_t = W_hp_t * Δt / 1000
    Ex_heat_out_t = Q_hp_h_opt[t] * carnot_heat * Δt / 1000
    Ex_D_hp += max(0, Ex_elec_in_t - Ex_heat_out_t)
end

# Total exergy destruction (internal irreversibility)
Ex_D_total = Ex_D_grid_plant + Ex_D_pv + Ex_D_wt + Ex_D_battery + Ex_D_furnace + Ex_D_hp + Ex_D_stc

# Total exergy loss (uncaptured resources - external)
Ex_loss_solar = Ex_loss_solar_pv  # Solar on uncovered rooftop
Ex_loss_total = Ex_loss_wind + Ex_loss_solar

println("\n┌─────────────────────────────────────────────────────────────────────┐")
println("│                    EXERGY DESTRUCTION BY COMPONENT                  │")
println("├─────────────────────────────────────────────────────────────────────┤")
@printf("│  Grid Power Plant (Brayton)        : %10.2f kWh_ex (%5.1f%%)    │\n", 
        Ex_D_grid_plant, 100*Ex_D_grid_plant/max(Ex_D_total,1e-6))
@printf("│  Solar PV                          : %10.2f kWh_ex (%5.1f%%)    │\n", 
        Ex_D_pv, 100*Ex_D_pv/max(Ex_D_total,1e-6))
@printf("│  Wind Turbine                      : %10.2f kWh_ex (%5.1f%%)    │\n", 
        Ex_D_wt, 100*Ex_D_wt/max(Ex_D_total,1e-6))
@printf("│  Battery Storage                   : %10.2f kWh_ex (%5.1f%%)    │\n", 
        Ex_D_battery, 100*Ex_D_battery/max(Ex_D_total,1e-6))
@printf("│  Gas Furnace                       : %10.2f kWh_ex (%5.1f%%)    │\n", 
        Ex_D_furnace, 100*Ex_D_furnace/max(Ex_D_total,1e-6))
@printf("│  Heat Pump                         : %10.2f kWh_ex (%5.1f%%)    │\n", 
        Ex_D_hp, 100*Ex_D_hp/max(Ex_D_total,1e-6))
@printf("│  Solar Thermal                     : %10.2f kWh_ex (%5.1f%%)    │\n", 
        Ex_D_stc, 100*Ex_D_stc/max(Ex_D_total,1e-6))
println("├─────────────────────────────────────────────────────────────────────┤")
@printf("│  TOTAL EXERGY DESTRUCTION          : %10.2f kWh_ex             │\n", Ex_D_total)
println("└─────────────────────────────────────────────────────────────────────┘")

println("\n┌─────────────────────────────────────────────────────────────────────┐")
println("│                    EXERGY LOSSES (Uncaptured Resources)             │")
println("├─────────────────────────────────────────────────────────────────────┤")
@printf("│  Solar (uncovered rooftop)         : %10.2f kWh_ex (%5.1f%%)    │\n", 
        Ex_loss_solar, 100*Ex_loss_solar/max(Ex_loss_total,1e-6))
@printf("│  Wind (not captured, passes through): %10.2f kWh_ex (%5.1f%%)    │\n", 
        Ex_loss_wind, 100*Ex_loss_wind/max(Ex_loss_total,1e-6))
println("├─────────────────────────────────────────────────────────────────────┤")
@printf("│  TOTAL EXERGY LOSS                 : %10.2f kWh_ex             │\n", Ex_loss_total)
println("└─────────────────────────────────────────────────────────────────────┘")


┌─────────────────────────────────────────────────────────────────────┐
│                    EXERGY DESTRUCTION BY COMPONENT                  │
├─────────────────────────────────────────────────────────────────────┤
│  Grid Power Plant (Brayton)        :    6337.46 kWh_ex (  9.8%)    │
│  Solar PV                          :   55818.29 kWh_ex ( 86.5%)    │
│  Wind Turbine                      :       0.00 kWh_ex (  0.0%)    │
│  Battery Storage                   :     449.74 kWh_ex (  0.7%)    │
│  Gas Furnace                       :       0.00 kWh_ex (  0.0%)    │
│  Heat Pump                         :    1957.91 kWh_ex (  3.0%)    │
│  Solar Thermal                     :       0.00 kWh_ex (  0.0%)    │
├─────────────────────────────────────────────────────────────────────┤
│  TOTAL EXERGY DESTRUCTION          :   64563.39 kWh_ex             │
└─────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────

In [141]:
# ============================================================
# 4. SYSTEM EFFICIENCY ANALYSIS
# ============================================================

# Energy inputs (First Law) - Extended boundary
En_in_grid_fuel = sum(P_grid_opt) / η_powerplant * Δt / 1000  # kWh (fuel energy at plant)
En_in_NG = sum(Q_furnace_opt) / η_furnace * Δt / 1000  # kWh (fuel LHV)
En_in_solar_pv = sum(G_solar[t] * Cap_pv_opt for t in t_set if G_solar[t] > 0) * Δt / 1000  # kWh
En_in_solar_stc = sum(G_solar[t] * Cap_stc_opt for t in t_set if G_solar[t] > 0) * Δt / 1000  # kWh
En_in_wind = Ex_in_wind  # Kinetic energy = exergy for wind
En_in_total = En_in_grid_fuel + En_in_NG + En_in_solar_pv + En_in_solar_stc + En_in_wind

# Energy outputs (First Law)
En_out_elec = sum(E_demand) * Δt / 1000  # kWh
En_out_heat = sum(Q_furnace_opt .+ Q_hp_h_opt .+ Q_stc_opt) * Δt / 1000  # kWh_th
En_out_cool = sum(Q_hp_c_opt) * Δt / 1000  # kWh_th
En_out_total = En_out_elec + En_out_heat + En_out_cool

# First Law Efficiency (Energy)
η_I = En_in_total > 0 ? En_out_total / En_in_total * 100 : 0.0

# Second Law Efficiency (Exergy)
η_II = Ex_in_total > 0 ? Ex_out_total / Ex_in_total * 100 : 0.0

# ============================================================
# EXERGY BALANCE VERIFICATION
# ============================================================
# Balance: Ex_in = Ex_out + Ex_D + Ex_loss
Ex_balance_check = Ex_in_total - Ex_out_total - Ex_D_total - Ex_loss_total
Ex_balance_error_pct = abs(Ex_balance_check) / Ex_in_total * 100

# Initial battery energy (accounts for small imbalance)
E_bat_initial = 0.5 * Cap_bat_opt / 1000  # kWh (starts at 50% SOC)

# Component-level exergy efficiencies
η_ex_pv = Ex_in_solar > 0 ? sum(P_pv_opt) * Δt / 1000 / Ex_in_solar * 100 : 0.0
η_ex_grid = 100 * η_powerplant  # Power plant efficiency
η_ex_bat = 100 * η_rt_bat  # Battery round-trip
η_ex_wind = Ex_in_wind > 0 ? Ex_captured_wind / Ex_in_wind * 100 : 0.0

# ============================================================
# 5. CO2 EMISSIONS ANALYSIS
# ============================================================

# Grid electricity emissions (from NG power plant)
# Emissions = electricity consumed × EF_grid
E_grid_total = sum(P_grid_opt) * Δt / 1000  # kWh
CO2_grid = E_grid_total * EF_grid  # kg CO2

# Furnace emissions (direct NG combustion)
# Emissions = fuel consumed × EF_NG
E_fuel_furnace = sum(Q_furnace_opt) / η_furnace * Δt / 1000  # kWh fuel
CO2_furnace = E_fuel_furnace * EF_NG  # kg CO2

# Total emissions
CO2_total = CO2_grid + CO2_furnace

# Emissions intensity (kg CO2 per kWh useful output)
CO2_intensity = En_out_total > 0 ? CO2_total / En_out_total : 0.0

# Avoided emissions from renewables (compared to 100% grid)
CO2_avoided_pv = sum(P_pv_opt) * Δt / 1000 * EF_grid  # kg CO2
CO2_avoided_wind = sum(P_wt_opt) * Δt / 1000 * EF_grid  # kg CO2
CO2_avoided_total = CO2_avoided_pv + CO2_avoided_wind

println("\n┌─────────────────────────────────────────────────────────────────────┐")
println("│                    SYSTEM EFFICIENCY ANALYSIS                       │")
println("├─────────────────────────────────────────────────────────────────────┤")
println("│                                                                     │")
println("│  ═══════════════════ FIRST LAW (ENERGY) ═══════════════════════    │")
@printf("│  Energy Input  (Fuel+Solar+Wind)    : %10.2f kWh              │\n", En_in_total)
@printf("│  Energy Output (Elec+Heat+Cool)     : %10.2f kWh              │\n", En_out_total)
@printf("│  FIRST LAW EFFICIENCY (η_I)         : %10.1f %%               │\n", η_I)
println("│                                                                     │")
println("│  ═══════════════════ SECOND LAW (EXERGY) ══════════════════════    │")
@printf("│  Exergy Input                       : %10.2f kWh_ex           │\n", Ex_in_total)
@printf("│  Exergy Output (useful)             : %10.2f kWh_ex           │\n", Ex_out_total)
@printf("│  Exergy Destruction                 : %10.2f kWh_ex           │\n", Ex_D_total)
@printf("│  Exergy Loss (uncaptured wind)      : %10.2f kWh_ex           │\n", Ex_loss_total)
@printf("│  SECOND LAW EFFICIENCY (η_II)       : %10.1f %%               │\n", η_II)
println("│                                                                     │")
println("│  ═══════════════════ EXERGY BALANCE CHECK ═════════════════════    │")
@printf("│  Ex_in - Ex_out - Ex_D - Ex_loss    : %10.2f kWh_ex           │\n", Ex_balance_check)
@printf("│  Initial Battery Energy (explains Δ): %10.2f kWh              │\n", E_bat_initial)
@printf("│  Balance Error                      : %10.2f %%               │\n", Ex_balance_error_pct)
println("│                                                                     │")
println("│  ═══════════════════ COMPONENT EFFICIENCIES ═══════════════════    │")
@printf("│  Grid Power Plant (Brayton)         : %10.1f %% (exergy)       │\n", η_ex_grid)
@printf("│  Solar PV                           : %10.1f %% (exergy)       │\n", η_ex_pv)
@printf("│  Wind Turbine                       : %10.1f %% (of available) │\n", η_ex_wind)
@printf("│  Battery Storage                    : %10.1f %% (round-trip)   │\n", η_ex_bat)
println("│                                                                     │")
println("└─────────────────────────────────────────────────────────────────────┘")

println("\n┌─────────────────────────────────────────────────────────────────────┐")
println("│                       CO₂ EMISSIONS ANALYSIS                        │")
println("├─────────────────────────────────────────────────────────────────────┤")
@printf("│  Grid Electricity (%.3f kg/kWh)    : %10.2f kg CO₂            │\n", EF_grid, CO2_grid)
@printf("│  Gas Furnace (%.3f kg/kWh_fuel)    : %10.2f kg CO₂            │\n", EF_NG, CO2_furnace)
println("├─────────────────────────────────────────────────────────────────────┤")
@printf("│  TOTAL EMISSIONS                    : %10.2f kg CO₂            │\n", CO2_total)
@printf("│  Emissions Intensity                : %10.3f kg CO₂/kWh        │\n", CO2_intensity)
println("├─────────────────────────────────────────────────────────────────────┤")
println("│  AVOIDED EMISSIONS (vs 100% grid):                                 │")
@printf("│    From Solar PV                    : %10.2f kg CO₂            │\n", CO2_avoided_pv)
@printf("│    From Wind                        : %10.2f kg CO₂            │\n", CO2_avoided_wind)
@printf("│    TOTAL AVOIDED                    : %10.2f kg CO₂            │\n", CO2_avoided_total)
println("└─────────────────────────────────────────────────────────────────────┘")

# Summary DataFrame
println("\n")
efficiency_summary = DataFrame(
    Metric = [
        "First Law Efficiency (η_I)",
        "Second Law Efficiency (η_II)", 
        "Exergy Destruction Ratio",
        "Renewable Exergy Fraction",
        "Exergy Balance Error",
        "CO₂ Emissions Total",
        "CO₂ Intensity"
    ],
    Value = [
        η_I,
        η_II,
        Ex_in_total > 0 ? Ex_D_total / Ex_in_total * 100 : 0.0,
        Ex_in_total > 0 ? (Ex_in_solar + Ex_in_wind) / Ex_in_total * 100 : 0.0,
        Ex_balance_error_pct,
        CO2_total,
        CO2_intensity
    ],
    Unit = ["%", "%", "%", "%", "%", "kg CO₂", "kg/kWh"],
    Interpretation = [
        η_I > 50 ? "Good" : (η_I > 30 ? "Moderate" : "Low"),
        η_II > 40 ? "Good" : (η_II > 20 ? "Moderate" : "Low"),
        Ex_D_total / max(Ex_in_total, 1e-6) * 100 < 70 ? "Acceptable" : "High irreversibility",
        (Ex_in_solar + Ex_in_wind) / max(Ex_in_total, 1e-6) * 100 > 50 ? "Renewable dominated" : "Fossil dependent",
        Ex_balance_error_pct < 5 ? "✓ Balance closes" : "Check calculations",
        CO2_total < 50 ? "Low emissions" : (CO2_total < 150 ? "Moderate" : "High"),
        CO2_intensity < 0.3 ? "Clean" : (CO2_intensity < 0.5 ? "Moderate" : "Carbon intensive")
    ]
)
println(efficiency_summary)


┌─────────────────────────────────────────────────────────────────────┐
│                    SYSTEM EFFICIENCY ANALYSIS                       │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  ═══════════════════ FIRST LAW (ENERGY) ═══════════════════════    │
│  Energy Input  (Fuel+Solar+Wind)    :   95829.55 kWh              │
│  Energy Output (Elec+Heat+Cool)     :   34236.84 kWh              │
│  FIRST LAW EFFICIENCY (η_I)         :       35.7 %               │
│                                                                     │
│  ═══════════════════ SECOND LAW (EXERGY) ══════════════════════    │
│  Exergy Input                       :  134625.59 kWh_ex           │
│  Exergy Output (useful)             :    7778.60 kWh_ex           │
│  Exergy Destruction                 :   64563.39 kWh_ex           │
│  Exergy Loss (uncaptured wind)      :   60290.88 kWh_ex           │
│  SECON

In [142]:
# ============================================================
# 3. EXERGY DESTRUCTION BY COMPONENT
# ============================================================

# Grid (Brayton cycle power plant) - destruction happens at power plant
# For imported electricity, the fuel exergy consumed at plant was:
# Ex_fuel_at_plant = P_grid / η_plant × ψ_NG, destruction = Ex_fuel - P_grid
Ex_D_grid_plant = sum(P_grid_opt) * (ψ_NG/η_powerplant - 1) * Δt / 1000  # kWh_ex

# ============================================================
# SOLAR PV - Destruction only on INSTALLED panel area
# Loss = solar on uncovered rooftop (passes through)
# ============================================================
Ex_D_pv = 0.0
Ex_loss_solar_pv = 0.0  # Solar on rooftop not covered by PV

for t in t_set
    G = G_solar[t]
    if G > 0
        # Solar exergy on PV panel area → some converted, rest is destruction
        Ex_solar_on_pv_t = ψ_solar_ex * G * Cap_pv_opt * Δt / 1000
        Ex_elec_out_t = P_pv_opt[t] * Δt / 1000
        Ex_D_pv += max(0, Ex_solar_on_pv_t - Ex_elec_out_t)
        
        # Solar exergy on uncovered rooftop (not PV, not STC) → passes through as LOSS
        A_uncovered = A_roof_max - Cap_pv_opt - Cap_stc_opt
        Ex_loss_solar_pv += ψ_solar_ex * G * max(0, A_uncovered) * Δt / 1000
    end
end

# ============================================================
# SOLAR THERMAL - Destruction only on INSTALLED collector area
# ============================================================
Ex_D_stc = 0.0
for t in t_set
    G = G_solar[t]
    if G > 0
        Ex_solar_on_stc_t = ψ_solar_ex * G * Cap_stc_opt * Δt / 1000
        Ex_heat_out_t = Q_stc_opt[t] * carnot_heat * Δt / 1000
        Ex_D_stc += max(0, Ex_solar_on_stc_t - Ex_heat_out_t)
    end
end

# ============================================================
# WIND TURBINE - Destruction only if turbine installed
# Loss = wind through swept area not captured (passes through)
# ============================================================
Ex_captured_wind = sum(P_wt_opt) * Δt / 1000  # Electricity captured from wind
Ex_D_wt = 0.0      # Destruction in turbine (conversion losses)
Ex_loss_wind = 0.0  # Uncaptured wind (passes through)

if Cap_wt_opt > 0
    # Wind turbine installed - some is captured, rest is destruction
    for t in t_set
        v = v_wind[t]
        if v > 0
            P_wind_available = 0.5 * ρ_air * A_swept_max * v^3  # W
            Ex_available_t = P_wind_available * Δt / 1000
            Ex_captured_t = P_wt_opt[t] * Δt / 1000
            Ex_D_wt += max(0, Ex_available_t - Ex_captured_t)
        end
    end
else
    # No turbine - all wind passes through uncaptured
    Ex_loss_wind = Ex_in_wind
end

# Battery - destruction from round-trip losses
Ex_D_battery = sum(P_ch_opt) * (1 - η_rt_bat) * Δt / 1000  # kWh_ex

# ============================================================
# PCM THERMAL STORAGE - Destruction from charge/discharge losses
# Exergy destruction = thermal exergy in - thermal exergy out
# ============================================================
# PCM operates at melting temperature T_m = 323.15 K (50°C)
T_pcm_m = 323.15  # PCM melting temperature [K]
carnot_pcm = 1 - T_ref / T_pcm_m  # Carnot factor for PCM temperature

# PCM round-trip efficiency
η_pcm_rt = η_pcm_ch * η_pcm_dch  # ≈ 0.855

# Exergy destruction in PCM = charging exergy loss + discharging exergy loss
# Charging: heat in at source temp → stored at PCM temp
# Discharging: stored at PCM temp → delivered at PCM temp (minus losses)
Ex_D_pcm = 0.0
for t in t_set
    # Charging: exergy in (from source, e.g., solar thermal or HP) - exergy stored
    # Assume source is at T_heat_delivery (50°C), so carnot factor is same
    Ex_charge_in_t = Q_pcm_ch_opt[t] * carnot_heat * Δt / 1000
    Ex_stored_t = Q_pcm_ch_opt[t] * η_pcm_ch * carnot_pcm * Δt / 1000
    Ex_D_pcm += max(0, Ex_charge_in_t - Ex_stored_t)
    
    # Discharging: exergy from storage - useful exergy delivered
    Ex_from_storage_t = Q_pcm_dch_opt[t] * carnot_pcm * Δt / 1000
    Ex_delivered_t = Q_pcm_dch_opt[t] * η_pcm_dch * carnot_heat * Δt / 1000
    Ex_D_pcm += max(0, Ex_from_storage_t - Ex_delivered_t)
end

# Furnace - destruction = chemical exergy - useful heat exergy
Ex_D_furnace = 0.0
for t in t_set
    Q_fuel_t = Q_furnace_opt[t] / η_furnace
    Ex_fuel_t = Q_fuel_t * ψ_NG * Δt / 1000
    Ex_heat_useful_t = Q_furnace_opt[t] * carnot_heat * Δt / 1000
    Ex_D_furnace += max(0, Ex_fuel_t - Ex_heat_useful_t)
end

# Heat Pump - destruction (electricity in - thermal exergy out)
Ex_D_hp = 0.0
for t in t_set
    T_outdoor = T_amb[t]
    ΔT_h = T_indoor_set - T_outdoor
    COP_h = max(1.0, 6.08 - 0.0941*abs(ΔT_h) + 0.000464*ΔT_h^2)
    
    W_hp_t = Q_hp_h_opt[t] / COP_h  # Electricity consumed
    Ex_elec_in_t = W_hp_t * Δt / 1000
    Ex_heat_out_t = Q_hp_h_opt[t] * carnot_heat * Δt / 1000
    Ex_D_hp += max(0, Ex_elec_in_t - Ex_heat_out_t)
end

# Total exergy destruction (internal irreversibility)
Ex_D_total = Ex_D_grid_plant + Ex_D_pv + Ex_D_wt + Ex_D_battery + Ex_D_pcm + Ex_D_furnace + Ex_D_hp + Ex_D_stc

# Total exergy loss (uncaptured resources - external)
Ex_loss_solar = Ex_loss_solar_pv  # Solar on uncovered rooftop
Ex_loss_total = Ex_loss_wind + Ex_loss_solar

println("\n┌─────────────────────────────────────────────────────────────────────┐")
println("│                    EXERGY DESTRUCTION BY COMPONENT                  │")
println("├─────────────────────────────────────────────────────────────────────┤")
@printf("│  Grid Power Plant (Brayton)        : %10.2f kWh_ex (%5.1f%%)    │\n", 
        Ex_D_grid_plant, 100*Ex_D_grid_plant/max(Ex_D_total,1e-6))
@printf("│  Solar PV                          : %10.2f kWh_ex (%5.1f%%)    │\n", 
        Ex_D_pv, 100*Ex_D_pv/max(Ex_D_total,1e-6))
@printf("│  Wind Turbine                      : %10.2f kWh_ex (%5.1f%%)    │\n", 
        Ex_D_wt, 100*Ex_D_wt/max(Ex_D_total,1e-6))
@printf("│  Battery Storage                   : %10.2f kWh_ex (%5.1f%%)    │\n", 
        Ex_D_battery, 100*Ex_D_battery/max(Ex_D_total,1e-6))
@printf("│  PCM Thermal Storage               : %10.2f kWh_ex (%5.1f%%)    │\n", 
        Ex_D_pcm, 100*Ex_D_pcm/max(Ex_D_total,1e-6))
@printf("│  Gas Furnace                       : %10.2f kWh_ex (%5.1f%%)    │\n", 
        Ex_D_furnace, 100*Ex_D_furnace/max(Ex_D_total,1e-6))
@printf("│  Heat Pump                         : %10.2f kWh_ex (%5.1f%%)    │\n", 
        Ex_D_hp, 100*Ex_D_hp/max(Ex_D_total,1e-6))
@printf("│  Solar Thermal                     : %10.2f kWh_ex (%5.1f%%)    │\n", 
        Ex_D_stc, 100*Ex_D_stc/max(Ex_D_total,1e-6))
println("├─────────────────────────────────────────────────────────────────────┤")
@printf("│  TOTAL EXERGY DESTRUCTION          : %10.2f kWh_ex             │\n", Ex_D_total)
println("└─────────────────────────────────────────────────────────────────────┘")

println("\n┌─────────────────────────────────────────────────────────────────────┐")
println("│                    EXERGY LOSSES (Uncaptured Resources)             │")
println("├─────────────────────────────────────────────────────────────────────┤")
@printf("│  Solar (uncovered rooftop)         : %10.2f kWh_ex (%5.1f%%)    │\n", 
        Ex_loss_solar, 100*Ex_loss_solar/max(Ex_loss_total,1e-6))
@printf("│  Wind (not captured, passes through): %10.2f kWh_ex (%5.1f%%)    │\n", 
        Ex_loss_wind, 100*Ex_loss_wind/max(Ex_loss_total,1e-6))
println("├─────────────────────────────────────────────────────────────────────┤")
@printf("│  TOTAL EXERGY LOSS                 : %10.2f kWh_ex             │\n", Ex_loss_total)
println("└─────────────────────────────────────────────────────────────────────┘")


┌─────────────────────────────────────────────────────────────────────┐
│                    EXERGY DESTRUCTION BY COMPONENT                  │
├─────────────────────────────────────────────────────────────────────┤
│  Grid Power Plant (Brayton)        :    6337.46 kWh_ex (  9.8%)    │
│  Solar PV                          :   55818.29 kWh_ex ( 86.3%)    │
│  Wind Turbine                      :       0.00 kWh_ex (  0.0%)    │
│  Battery Storage                   :     449.74 kWh_ex (  0.7%)    │
│  PCM Thermal Storage               :      90.80 kWh_ex (  0.1%)    │
│  Gas Furnace                       :       0.00 kWh_ex (  0.0%)    │
│  Heat Pump                         :    1957.91 kWh_ex (  3.0%)    │
│  Solar Thermal                     :       0.00 kWh_ex (  0.0%)    │
├─────────────────────────────────────────────────────────────────────┤
│  TOTAL EXERGY DESTRUCTION          :   64654.19 kWh_ex             │
└─────────────────────────────────────────────────────────────────────┘


In [143]:
# ============================================================
# COST BREAKDOWN
# ============================================================

# Operating costs
grid_cost_total = sum(P_grid_opt) * c_grid * Δt
ng_cost_total = sum(Q_furnace_opt) / η_furnace * c_ng * Δt
operating_total = grid_cost_total + ng_cost_total

# Capital costs (annualized for simulation period)
cap_cost_pv = ann_pv * Cap_pv_opt * T
cap_cost_wt = ann_wt * Cap_wt_opt * T
cap_cost_bat = ann_bat * Cap_bat_opt * T
cap_cost_stc = ann_stc * Cap_stc_opt * T
capital_total = cap_cost_pv + cap_cost_wt + cap_cost_bat + cap_cost_stc

println("\n" * "="^60)
println("              COST BREAKDOWN ($(T) hours)")
println("="^60)

println("\nOPERATING COSTS (existing infrastructure):")
cost_op = DataFrame(
    Source = ["Grid Electricity", "Natural Gas", "Operating Total"],
    Cost_USD = [grid_cost_total, ng_cost_total, operating_total]
)
println(cost_op)

println("\nCAPITAL COSTS (new investments, annualized):")
cost_cap = DataFrame(
    Technology = ["Solar PV", "Wind Turbine", "Battery", "Solar Thermal", "Capital Total"],
    Size = [Cap_pv_opt, Cap_wt_opt/1000, Cap_bat_opt/1000, Cap_stc_opt, NaN],
    Unit = ["m²", "kW", "kWh", "m²", ""],
    Ann_Cost = [cap_cost_pv, cap_cost_wt, cap_cost_bat, cap_cost_stc, capital_total]
)
println(cost_cap)

println("\n" * "─"^60)
@printf("TOTAL COST (Operating + Capital): \$%.2f\n", operating_total + capital_total)
@printf("  - Operating: \$%.2f (%.1f%%)\n", operating_total, 100*operating_total/(operating_total+capital_total))
@printf("  - Capital:   \$%.2f (%.1f%%)\n", capital_total, 100*capital_total/(operating_total+capital_total))


              COST BREAKDOWN (8760 hours)

OPERATING COSTS (existing infrastructure):
3×2 DataFrame
 Row │ Source            Cost_USD 
     │ String            Float64  
─────┼────────────────────────────
   1 │ Grid Electricity    1728.4
   2 │ Natural Gas            0.0
   3 │ Operating Total     1728.4

CAPITAL COSTS (new investments, annualized):
5×4 DataFrame
 Row │ Technology     Size      Unit    Ann_Cost 
     │ String         Float64   String  Float64  
─────┼───────────────────────────────────────────
   1 │ Solar PV        35.9067  m²       1950.94
   2 │ Wind Turbine     0.0     kW          0.0
   3 │ Battery         10.0993  kWh      1733.87
   4 │ Solar Thermal   -0.0     m²         -0.0
   5 │ Capital Total  NaN                3684.81

────────────────────────────────────────────────────────────
TOTAL COST (Operating + Capital): $5413.21
  - Operating: $1728.40 (31.9%)
  - Capital:   $3684.81 (68.1%)


## Tests

In [144]:
# ============================================================
# COST BREAKDOWN
# ============================================================

# Operating costs
grid_cost_total = sum(P_grid_opt) * c_grid * Δt
ng_cost_total = sum(Q_furnace_opt) / η_furnace * c_ng * Δt
operating_total = grid_cost_total + ng_cost_total

# Capital costs (annualized for simulation period)
cap_cost_pv = ann_pv * Cap_pv_opt * T
cap_cost_wt = ann_wt * Cap_wt_opt * T
cap_cost_bat = ann_bat * Cap_bat_opt * T
cap_cost_stc = ann_stc * Cap_stc_opt * T
cap_cost_pcm = ann_pcm * Cap_pcm_opt * T
capital_total = cap_cost_pv + cap_cost_wt + cap_cost_bat + cap_cost_stc + cap_cost_pcm

println("\n" * "="^60)
println("              COST BREAKDOWN ($(T) hours)")
println("="^60)

println("\nOPERATING COSTS (existing infrastructure):")
cost_op = DataFrame(
    Source = ["Grid Electricity", "Natural Gas", "Operating Total"],
    Cost_USD = [grid_cost_total, ng_cost_total, operating_total]
)
println(cost_op)

println("\nCAPITAL COSTS (new investments, annualized):")
cost_cap = DataFrame(
    Technology = ["Solar PV", "Wind Turbine", "Battery", "Solar Thermal", "PCM Storage", "Capital Total"],
    Size = [Cap_pv_opt, Cap_wt_opt/1000, Cap_bat_opt/1000, Cap_stc_opt, Cap_pcm_opt/1000, NaN],
    Unit = ["m²", "kW", "kWh", "m²", "kWh_th", ""],
    Ann_Cost = [cap_cost_pv, cap_cost_wt, cap_cost_bat, cap_cost_stc, cap_cost_pcm, capital_total]
)
println(cost_cap)

println("\n" * "─"^60)
@printf("TOTAL COST (Operating + Capital): \$%.2f\n", operating_total + capital_total)
@printf("  - Operating: \$%.2f (%.1f%%)\n", operating_total, 100*operating_total/(operating_total+capital_total))
@printf("  - Capital:   \$%.2f (%.1f%%)\n", capital_total, 100*capital_total/(operating_total+capital_total))


              COST BREAKDOWN (8760 hours)

OPERATING COSTS (existing infrastructure):
3×2 DataFrame
 Row │ Source            Cost_USD 
     │ String            Float64  
─────┼────────────────────────────
   1 │ Grid Electricity    1728.4
   2 │ Natural Gas            0.0
   3 │ Operating Total     1728.4

CAPITAL COSTS (new investments, annualized):
6×4 DataFrame
 Row │ Technology     Size      Unit    Ann_Cost 
     │ String         Float64   String  Float64  
─────┼───────────────────────────────────────────
   1 │ Solar PV        35.9067  m²      1950.94
   2 │ Wind Turbine     0.0     kW         0.0
   3 │ Battery         10.0993  kWh     1733.87
   4 │ Solar Thermal   -0.0     m²        -0.0
   5 │ PCM Storage     57.1439  kWh_th   174.607
   6 │ Capital Total  NaN               3859.41

────────────────────────────────────────────────────────────
TOTAL COST (Operating + Capital): $5587.81
  - Operating: $1728.40 (30.9%)
  - Capital:   $3859.41 (69.1%)


## Export Results to Excel

Export comprehensive optimization results to an Excel file with multiple sheets:
1. **Summary** - Model run information, optimal capacities, costs, efficiencies
2. **Hourly_Dispatch** - Time-series data for all dispatch variables
3. **Exergy_Analysis** - Detailed exergy balance and destruction by component
4. **Emissions** - CO₂ emissions breakdown

In [145]:
using XLSX
using Dates

# ============================================================
# EXPORT RESULTS TO EXCEL
# ============================================================

# Generate output filename with timestamp
timestamp = Dates.format(now(), "yyyy-mm-dd_HHMMSS")
output_file = "optimization_results_$(timestamp).xlsx"

# ============================================================
# SHEET 1: SUMMARY
# ============================================================
summary_data = DataFrame(
    Category = String[],
    Parameter = String[],
    Value = Any[],
    Unit = String[]
)

# Model Run Information
push!(summary_data, ("Model Run", "Timestamp", string(now()), ""))
push!(summary_data, ("Model Run", "Solver Status", string(termination_status(model)), ""))
push!(summary_data, ("Model Run", "Objective Value", round(objective_value(model), digits=4), "\$"))
push!(summary_data, ("Model Run", "Time Steps", T, "hours"))
push!(summary_data, ("Model Run", "Time Step Duration", Δt, "hours"))

# Optimal Capacities
push!(summary_data, ("Optimal Capacities", "Solar PV Area", round(Cap_pv_opt, digits=2), "m²"))
push!(summary_data, ("Optimal Capacities", "Solar PV Power", round(Cap_pv_opt * 0.20, digits=2), "kW"))
push!(summary_data, ("Optimal Capacities", "Wind Turbine", round(Cap_wt_opt/1000, digits=2), "kW"))
push!(summary_data, ("Optimal Capacities", "Battery Storage", round(Cap_bat_opt/1000, digits=2), "kWh"))
push!(summary_data, ("Optimal Capacities", "Solar Thermal Area", round(Cap_stc_opt, digits=2), "m²"))
push!(summary_data, ("Optimal Capacities", "PCM Thermal Storage", round(Cap_pcm_opt/1000, digits=2), "kWh_th"))
push!(summary_data, ("Optimal Capacities", "Total Rooftop Used", round(Cap_pv_opt + Cap_stc_opt, digits=2), "m²"))

# Capital Costs
total_investment = Cap_pv_opt * 0.20 * cost_pv_per_kW + Cap_wt_opt/1000 * cost_wt_per_kW + 
                   Cap_bat_opt/1000 * cost_bat_per_kWh + Cap_stc_opt * cost_stc_per_m2 +
                   Cap_pcm_opt/1000 * cost_pcm_per_kWh
push!(summary_data, ("Capital Costs", "Solar PV Investment", round(Cap_pv_opt * 0.20 * cost_pv_per_kW, digits=2), "\$"))
push!(summary_data, ("Capital Costs", "Wind Turbine Investment", round(Cap_wt_opt/1000 * cost_wt_per_kW, digits=2), "\$"))
push!(summary_data, ("Capital Costs", "Battery Investment", round(Cap_bat_opt/1000 * cost_bat_per_kWh, digits=2), "\$"))
push!(summary_data, ("Capital Costs", "Solar Thermal Investment", round(Cap_stc_opt * cost_stc_per_m2, digits=2), "\$"))
push!(summary_data, ("Capital Costs", "PCM Storage Investment", round(Cap_pcm_opt/1000 * cost_pcm_per_kWh, digits=2), "\$"))
push!(summary_data, ("Capital Costs", "Total Investment", round(total_investment, digits=2), "\$"))

# Operating & Total Costs
push!(summary_data, ("Costs (Simulation Period)", "Grid Electricity Cost", round(grid_cost_total, digits=4), "\$"))
push!(summary_data, ("Costs (Simulation Period)", "Natural Gas Cost", round(ng_cost_total, digits=4), "\$"))
push!(summary_data, ("Costs (Simulation Period)", "Total Operating Cost", round(operating_total, digits=4), "\$"))
push!(summary_data, ("Costs (Simulation Period)", "Annualized Capital Cost", round(capital_total, digits=4), "\$"))
push!(summary_data, ("Costs (Simulation Period)", "Total Cost", round(operating_total + capital_total, digits=4), "\$"))

# Energy Summary
push!(summary_data, ("Energy Summary", "Total Grid Import", round(sum(P_grid_opt) * Δt / 1000, digits=2), "kWh"))
push!(summary_data, ("Energy Summary", "Total PV Generation", round(sum(P_pv_opt) * Δt / 1000, digits=2), "kWh"))
push!(summary_data, ("Energy Summary", "Total Wind Generation", round(sum(P_wt_opt) * Δt / 1000, digits=2), "kWh"))
push!(summary_data, ("Energy Summary", "Total Furnace Heat", round(sum(Q_furnace_opt) * Δt / 1000, digits=2), "kWh_th"))
push!(summary_data, ("Energy Summary", "Total HP Heating", round(sum(Q_hp_h_opt) * Δt / 1000, digits=2), "kWh_th"))
push!(summary_data, ("Energy Summary", "Total HP Cooling", round(sum(Q_hp_c_opt) * Δt / 1000, digits=2), "kWh_th"))
push!(summary_data, ("Energy Summary", "Total Solar Thermal", round(sum(Q_stc_opt) * Δt / 1000, digits=2), "kWh_th"))
push!(summary_data, ("Energy Summary", "Total PCM Discharge", round(sum(Q_pcm_dch_opt) * Δt / 1000, digits=2), "kWh_th"))
push!(summary_data, ("Energy Summary", "Battery Throughput", round(sum(P_ch_opt) * Δt / 1000, digits=2), "kWh"))

# Efficiency Metrics
push!(summary_data, ("Efficiency", "First Law Efficiency (η_I)", round(η_I, digits=2), "%"))
push!(summary_data, ("Efficiency", "Second Law Efficiency (η_II)", round(η_II, digits=2), "%"))
push!(summary_data, ("Efficiency", "Exergy Destruction Ratio", round(Ex_D_total / max(Ex_in_total, 1e-6) * 100, digits=2), "%"))
push!(summary_data, ("Efficiency", "Renewable Exergy Fraction", round((Ex_in_solar + Ex_in_wind) / max(Ex_in_total, 1e-6) * 100, digits=2), "%"))

# Emissions
push!(summary_data, ("Emissions", "Grid CO₂ Emissions", round(CO2_grid, digits=3), "kg CO₂"))
push!(summary_data, ("Emissions", "Furnace CO₂ Emissions", round(CO2_furnace, digits=3), "kg CO₂"))
push!(summary_data, ("Emissions", "Total CO₂ Emissions", round(CO2_total, digits=3), "kg CO₂"))
push!(summary_data, ("Emissions", "CO₂ Intensity", round(CO2_intensity, digits=4), "kg CO₂/kWh"))
push!(summary_data, ("Emissions", "Avoided Emissions (PV)", round(CO2_avoided_pv, digits=3), "kg CO₂"))
push!(summary_data, ("Emissions", "Avoided Emissions (Wind)", round(CO2_avoided_wind, digits=3), "kg CO₂"))
push!(summary_data, ("Emissions", "Total Avoided Emissions", round(CO2_avoided_total, digits=3), "kg CO₂"))

# ============================================================
# SHEET 2: HOURLY DISPATCH
# ============================================================

# Calculate hourly COP values for export
COP_h_hourly = zeros(T)
COP_c_hourly = zeros(T)
for t in 1:T
    T_outdoor = T_amb[t]
    ΔT_h = T_indoor_set - T_outdoor
    ΔT_c = T_indoor_set - T_outdoor
    COP_h_hourly[t] = max(1.0, 6.08 - 0.0941*abs(ΔT_h) + 0.000464*ΔT_h^2)
    COP_c_hourly[t] = max(1.0, 5.08 - 0.0941*abs(ΔT_c) + 0.000464*ΔT_c^2)
end

# Calculate HP electricity consumption
W_hp_h_hourly = collect(Q_hp_h_opt) ./ COP_h_hourly
W_hp_c_hourly = collect(Q_hp_c_opt) ./ COP_c_hourly

hourly_data = DataFrame(
    Hour = collect(1:T),
    Electricity_Demand_W = E_demand,
    Heating_Demand_W = Q_heat_demand,
    Cooling_Demand_W = Q_cool_demand,
    Solar_Irradiance_W_m2 = G_solar,
    Wind_Speed_m_s = v_wind,
    Ambient_Temp_C = T_amb .- 273.15,
    P_Grid_W = collect(P_grid_opt),
    P_PV_W = collect(P_pv_opt),
    P_Wind_W = collect(P_wt_opt),
    P_Battery_Charge_W = collect(P_ch_opt),
    P_Battery_Discharge_W = collect(P_dch_opt),
    Q_Furnace_W = collect(Q_furnace_opt),
    Q_HP_Heating_W = collect(Q_hp_h_opt),
    Q_HP_Cooling_W = collect(Q_hp_c_opt),
    Q_Solar_Thermal_W = collect(Q_stc_opt),
    Q_PCM_Charge_W = collect(Q_pcm_ch_opt),
    Q_PCM_Discharge_W = collect(Q_pcm_dch_opt),
    Battery_SOC_Wh = collect(SOC_opt),
    PCM_Energy_Wh = collect(E_pcm_opt),
    PV_Efficiency = η_pv_t,
    Wind_Capacity_Factor = CF_wt_t,
    STC_Efficiency = η_stc_t,
    HP_COP_Heating = COP_h_hourly,
    HP_COP_Cooling = COP_c_hourly,
    W_HP_Heating_W = W_hp_h_hourly,
    W_HP_Cooling_W = W_hp_c_hourly
)

# ============================================================
# SHEET 3: EXERGY ANALYSIS
# ============================================================
exergy_data = DataFrame(
    Category = String[],
    Component = String[],
    Value_kWh_ex = Float64[],
    Percentage = Float64[]
)

# Exergy Inputs
push!(exergy_data, ("Exergy Input", "Grid Fuel (at Power Plant)", round(Ex_in_grid_fuel, digits=4), round(100*Ex_in_grid_fuel/max(Ex_in_total,1e-6), digits=2)))
push!(exergy_data, ("Exergy Input", "Natural Gas Direct", round(Ex_in_NG, digits=4), round(100*Ex_in_NG/max(Ex_in_total,1e-6), digits=2)))
push!(exergy_data, ("Exergy Input", "Solar Radiation (full rooftop)", round(Ex_in_solar, digits=4), round(100*Ex_in_solar/max(Ex_in_total,1e-6), digits=2)))
push!(exergy_data, ("Exergy Input", "Wind Kinetic", round(Ex_in_wind, digits=4), round(100*Ex_in_wind/max(Ex_in_total,1e-6), digits=2)))
push!(exergy_data, ("Exergy Input", "TOTAL INPUT", round(Ex_in_total, digits=4), 100.0))

# Exergy Outputs
push!(exergy_data, ("Exergy Output", "Electrical Load", round(Ex_out_elec, digits=4), round(100*Ex_out_elec/max(Ex_out_total,1e-6), digits=2)))
push!(exergy_data, ("Exergy Output", "Heating Exergy", round(Ex_out_heat, digits=4), round(100*Ex_out_heat/max(Ex_out_total,1e-6), digits=2)))
push!(exergy_data, ("Exergy Output", "Cooling Exergy", round(Ex_out_cool, digits=4), round(100*Ex_out_cool/max(Ex_out_total,1e-6), digits=2)))
push!(exergy_data, ("Exergy Output", "TOTAL OUTPUT", round(Ex_out_total, digits=4), 100.0))

# Exergy Destruction
push!(exergy_data, ("Exergy Destruction", "Grid Power Plant", round(Ex_D_grid_plant, digits=4), round(100*Ex_D_grid_plant/max(Ex_D_total,1e-6), digits=2)))
push!(exergy_data, ("Exergy Destruction", "Solar PV", round(Ex_D_pv, digits=4), round(100*Ex_D_pv/max(Ex_D_total,1e-6), digits=2)))
push!(exergy_data, ("Exergy Destruction", "Wind Turbine", round(Ex_D_wt, digits=4), round(100*Ex_D_wt/max(Ex_D_total,1e-6), digits=2)))
push!(exergy_data, ("Exergy Destruction", "Battery Storage", round(Ex_D_battery, digits=4), round(100*Ex_D_battery/max(Ex_D_total,1e-6), digits=2)))
push!(exergy_data, ("Exergy Destruction", "PCM Thermal Storage", round(Ex_D_pcm, digits=4), round(100*Ex_D_pcm/max(Ex_D_total,1e-6), digits=2)))
push!(exergy_data, ("Exergy Destruction", "Gas Furnace", round(Ex_D_furnace, digits=4), round(100*Ex_D_furnace/max(Ex_D_total,1e-6), digits=2)))
push!(exergy_data, ("Exergy Destruction", "Heat Pump", round(Ex_D_hp, digits=4), round(100*Ex_D_hp/max(Ex_D_total,1e-6), digits=2)))
push!(exergy_data, ("Exergy Destruction", "Solar Thermal", round(Ex_D_stc, digits=4), round(100*Ex_D_stc/max(Ex_D_total,1e-6), digits=2)))
push!(exergy_data, ("Exergy Destruction", "TOTAL DESTRUCTION", round(Ex_D_total, digits=4), 100.0))

# Exergy Losses
push!(exergy_data, ("Exergy Loss", "Uncaptured Solar (uncovered rooftop)", round(Ex_loss_solar, digits=4), round(100*Ex_loss_solar/max(Ex_loss_total,1e-6), digits=2)))
push!(exergy_data, ("Exergy Loss", "Uncaptured Wind", round(Ex_loss_wind, digits=4), round(100*Ex_loss_wind/max(Ex_loss_total,1e-6), digits=2)))
push!(exergy_data, ("Exergy Loss", "TOTAL LOSS", round(Ex_loss_total, digits=4), 100.0))

# Balance Check
push!(exergy_data, ("Balance Check", "Ex_in - Ex_out - Ex_D - Ex_loss", round(Ex_balance_check, digits=4), round(Ex_balance_error_pct, digits=2)))

# ============================================================
# SHEET 4: EMISSIONS ANALYSIS
# ============================================================
emissions_data = DataFrame(
    Category = String[],
    Source = String[],
    Value = Float64[],
    Unit = String[]
)

push!(emissions_data, ("Direct Emissions", "Grid Electricity", round(CO2_grid, digits=4), "kg CO₂"))
push!(emissions_data, ("Direct Emissions", "Gas Furnace", round(CO2_furnace, digits=4), "kg CO₂"))
push!(emissions_data, ("Direct Emissions", "Total", round(CO2_total, digits=4), "kg CO₂"))
push!(emissions_data, ("Intensity", "CO₂ per kWh Output", round(CO2_intensity, digits=4), "kg CO₂/kWh"))
push!(emissions_data, ("Avoided Emissions", "From Solar PV", round(CO2_avoided_pv, digits=4), "kg CO₂"))
push!(emissions_data, ("Avoided Emissions", "From Wind", round(CO2_avoided_wind, digits=4), "kg CO₂"))
push!(emissions_data, ("Avoided Emissions", "Total Avoided", round(CO2_avoided_total, digits=4), "kg CO₂"))
push!(emissions_data, ("Emission Factors", "Natural Gas (EF_NG)", EF_NG, "kg CO₂/kWh_fuel"))
push!(emissions_data, ("Emission Factors", "Grid Electricity (EF_grid)", round(EF_grid, digits=4), "kg CO₂/kWh_elec"))

# ============================================================
# WRITE TO EXCEL FILE
# ============================================================
XLSX.writetable(output_file, 
    Summary = (collect(DataFrames.eachcol(summary_data)), DataFrames.names(summary_data)),
    Hourly_Dispatch = (collect(DataFrames.eachcol(hourly_data)), DataFrames.names(hourly_data)),
    Exergy_Analysis = (collect(DataFrames.eachcol(exergy_data)), DataFrames.names(exergy_data)),
    Emissions = (collect(DataFrames.eachcol(emissions_data)), DataFrames.names(emissions_data))
)

println("\n" * "="^60)
println("         RESULTS EXPORTED TO EXCEL")
println("="^60)
println("Output file: $(output_file)")
println("\nSheets created:")
println("  1. Summary          - Model info, capacities, costs, efficiencies")
println("  2. Hourly_Dispatch  - $(T) rows of time-series dispatch data")
println("  3. Exergy_Analysis  - Exergy balance and destruction breakdown")
println("  4. Emissions        - CO₂ emissions analysis")
println("="^60)


         RESULTS EXPORTED TO EXCEL
Output file: optimization_results_2026-02-04_114155.xlsx

Sheets created:
  1. Summary          - Model info, capacities, costs, efficiencies
  2. Hourly_Dispatch  - 8760 rows of time-series dispatch data
  3. Exergy_Analysis  - Exergy balance and destruction breakdown
  4. Emissions        - CO₂ emissions analysis
